### 加载数据集

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from utils.dataset_utils import get_classes_indexes_counts
import scipy.io as sio  # 从.mat文件中读取数据集
import numpy as np

################################################################加载数据集################################################
# 数据集
mat_data = sio.loadmat('../../data/dataset/USPS.mat')
# 提取变量
dataset_x = mat_data['X']
dataset_y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
# 显示数据集分布
print("特征数据:", dataset_x.shape)
print("label:", dataset_y.shape)

# One-hot encode target variable 强制将类别转换为0，1，2，3......
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(dataset_y.reshape(-1, 1))

# 统计每个类别的个数，dataset_y.max()+1是类别的个数
classes, counts = get_classes_indexes_counts(dataset_y)
print("每种类别的数量：", counts)

特征数据: (9298, 256)
label: (9298,)
每种类别的数量： [   0 1553 1269  929  824  852  716  834  792  708  821]


###  划分数据集

In [5]:
from sklearn.preprocessing import StandardScaler

x_train, x_test, y_train, y_test = train_test_split(dataset_x, y_onehot, test_size=0.3, random_state=42)

# Standardize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

# 显示数据集分布
print("特征数据:", x_train.shape)
print("label:", y_train.shape)

# 统计每个类别的个数 np.argmax(y_train, axis=1) Convert one-hot encoded test labels back to single class labels
classes_train, counts_train = get_classes_indexes_counts(np.argmax(y_train, axis=1))
print("训练集每种类别的数量：", counts_train)

classes_test, counts_test = get_classes_indexes_counts(np.argmax(y_test, axis=1))
print("测试集每种类别的数量：", counts_test)

特征数据: (6508, 256)
label: (6508, 10)
训练集每种类别的数量： [1116  884  645  569  593  498  574  548  497  584]
测试集每种类别的数量： [437 385 284 255 259 218 260 244 211 237]


### 平衡数据集的构建
1.首先找出类别数量最小的类
2.取最小类别数量的90%，作为平衡数据集的各类别原始数量
3.在每个类别中随机抽取实例构成初始数据集

In [6]:
# 确定每个类别的数量
num_instances = int(counts_train.min() * 0.9)  # 向下取整
print("最小数量:", num_instances)

# 在每个类别中随机的选择该数量的实例的索引
balanced_classes = np.array([])
for indexes in classes_train:
    random_selecte_indices = np.random.choice(indexes, size=num_instances, replace=False)
    balanced_classes = np.hstack((balanced_classes, random_selecte_indices))
balanced_classes = np.sort(balanced_classes).astype(int)

# 得到平衡的数据集
balanced_dataset_x = []
balanced_dataset_y = []
for index in balanced_classes:
    balanced_dataset_x.append(x_train[index])
    balanced_dataset_y.append(y_train[index, :])
balanced_dataset_x = np.array(balanced_dataset_x)
balanced_dataset_y = np.array(balanced_dataset_y).astype(int)

# 显示数据集分布
print("平衡的数据集的特征数据:", balanced_dataset_x.shape)
print("label:", balanced_dataset_y.shape)

# 统计每个类别的个数
classes_balanced_dataset, counts_balanced_dataset = get_classes_indexes_counts(np.argmax(balanced_dataset_y, axis=1))
print("平衡的数据集中每种类别的数量：", counts_balanced_dataset)

最小数量: 447
平衡的数据集的特征数据: (4470, 256)
label: (4470, 10)
平衡的数据集中每种类别的数量： [447 447 447 447 447 447 447 447 447 447]


### 多目标评价

In [7]:
from scipy.stats import gmean
import math
from sklearn.metrics import confusion_matrix, precision_score, roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt


##########################由个体得到选择的实例子集的索引###########################
def get_indices(individual):
    '''
    :param individual: individual（用实值进行编码）
    :return: 被选择实例的索引
    '''
    individual = np.round(individual)  # 数据范围在0-1之间，转化成int的同时会舍去小数部分，从而将个体映射到0-1编码
    indices = np.where(individual == 1)  # 1代表选择该实例，返回值是tuple，tuple[0]取元组中的第一个元素
    return indices[0]


###########################获取实例子集############################
def get_subset(individual):
    '''
    :param individual: 
    :return: 实例子集
    '''
    indices = get_indices(individual)
    x_sub = balanced_dataset_x[indices, :]
    y_sub = balanced_dataset_y[indices, :]
    return x_sub, y_sub


##########################适应度函数（PPV和PFC，为主要、次要指标）#################################
def fitness_function(individual):
    ######################PPV#######################
    # 使用训练数据进行预测
    index_pred = individual.mlp.predict(x_test)  # 计算accuracy、PPV
    index_pred_proba = individual.mlp.predict_proba(x_test)  # 计算mAUC

    # Convert one-hot encoded test labels back to single class labels
    y_test_labels = np.argmax(y_test, axis=1)
    y_pred_labels = np.argmax(index_pred, axis=1)
    _, counts = get_classes_indexes_counts(y_test_labels)
    #print("类型数量",len(counts))
    # 计算每个类别的 Precision
    class_precisions = precision_score(y_test_labels, y_pred_labels, average=None)
    #print("每个类别的PPV：", class_precisions)
    geometric_mean = gmean(class_precisions)
    ######################PFC#######################
    # 7. 计算 ROC AUC（ovo+macro）
    auc_ovo_macro = roc_auc_score(y_test, index_pred_proba, multi_class="ovo", average="macro")
    return round(geometric_mean, 4), round(auc_ovo_macro, 4)



### NSGA-II

In [8]:
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode
from instance_selection.nsga_2.genetic_operator import selNSGA2, mutate_binary_inversion, selTournamentDCD
import warnings

warnings.filterwarnings("ignore")  # 忽略警告

from sklearn.neural_network import MLPClassifier

import array
import random
import numpy
from deap import base
from deap import creator
from deap import tools

# 最大化评价目标
creator.create("FitnessMaxAndMax", base.Fitness, weights=(1.0, 1.0))
'''
fitness:适应度：Gmean和mAUC
class_accuracies：分类器对每个类的分类准确度
PPVs：PPV也即Precision,PPVs表示对每个类的分类精度
PFC：每个分类器的成对故障信用，用于评估分类器集合的多样性

'''
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMaxAndMax, pfc=None, mlp=None)
toolbox = base.Toolbox()

BOUND_LOW, BOUND_UP = 0.0, 1.0

NDIM = num_instances
# 二进制编码
toolbox.register("attr_binary", random.randint, 0, 1)  # 0-1编码
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_binary, n=num_instances)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# toolbox.register("evaluate", benchmarks.zdt1)
toolbox.register("evaluate", fitness_function)
# toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0)
# 单点交叉
toolbox.register("mate", tools.cxOnePoint)
# 二进制突变
toolbox.register("mutate", mutate_binary_inversion)
toolbox.register("select", selNSGA2,x_test=x_test,y_test=y_test)
# toolbox.register("select", tools.selTournament, tournsize=3)

init_mlp = MLPClassifier(hidden_layer_sizes=(20,), max_iter=800, random_state=42)


def main(seed=None):
    random.seed(seed)

    NGEN = 5
    MU = 40
    CXPB = 1.0
    MR = 0.2

    ####################################迭代过程的记录###########################
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    # stats.register("avg", numpy.mean, axis=0)
    # stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "min", "max"

    ####################################种群的初始化###########################
    pop = toolbox.population(n=MU)

    ####################################计算初始种群的适应度###########################
    ensembles = []  # 当前每个个体对应的mlp模型
    base_estimators = []  # 基学习器
    save_ensembles = []  # 存储每个个体对应的mlp模型
    pop_x_sub = []  # 当前每个个体的实例选择的特征数据
    pop_y_sub = []  # 当前每个个体对应的实例选择的lable
    # 对于每个个体都训练得到一个mlp模型
    for i in range(len(pop)):
        mlp = MLPClassifier(hidden_layer_sizes=(20,), max_iter=800, random_state=42)
        x_sub, y_sub = get_subset(pop[i])
        mlp.fit(x_sub, y_sub)
        ensembles.append(mlp)
        pop_x_sub.append(x_sub)
        pop_y_sub.append(y_sub)
        pop[i].mlp = mlp

    # 由mlp模型得到个体的适应度
    for i in range(len(pop)):
        pop[i].fitness.values = toolbox.evaluate(pop[i])

    #################################计算PFC并进行非支配排序#########################################
    # 计算PFC并进行非支配排序 PFC代替拥挤距离
    pop = toolbox.select(pop, len(pop))

    record = stats.compile(pop)
    logbook.record(gen=0, evals=len(pop), **record)
    print(logbook.stream)
    ####################################种群的迭代###########################
    for gen in range(1, NGEN):
        # 选择
        offspring = selTournamentDCD(pop, len(pop))
        offspring = [toolbox.clone(ind) for ind in offspring]

        # 交叉
        for i in range(0, len(offspring) - 1, 2):
            if random.random() <= CXPB:
                offspring[i], offspring[i + 1] = toolbox.mate(offspring[i], offspring[i + 1])
            # 突变
            offspring[i] = toolbox.mutate(offspring[i])[0]
            offspring[i + 1] = toolbox.mutate(offspring[i + 1])[0]
            del offspring[i].fitness.values, offspring[i + 1].fitness.values

        # 计算新的种群适应度 
        ensembles.clear()
        pop_x_sub.clear()
        pop_y_sub.clear()
        for i in range(len(offspring)):
            mlp = MLPClassifier(hidden_layer_sizes=(20,), max_iter=1000, random_state=42)
            x_sub, y_sub = get_subset(offspring[i])
            mlp.fit(x_sub, y_sub)
            ensembles.append(mlp)
            pop_x_sub.append(x_sub)
            pop_y_sub.append(y_sub)
            offspring[i].mlp = mlp
        for i in range(len(offspring)):
            offspring[i].fitness.values = toolbox.evaluate(offspring[i])

        # Select the next generation population
        pop = toolbox.select(pop + offspring, MU)
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(pop), **record)
        print(logbook.stream)
    for ind in pop :
        save_ensembles.append(ind.mlp)
    return pop, logbook, save_ensembles


def vote_ensembles(save_ensembles):
    y_pred_labels_ensembles=[]
    y_test_labels = np.argmax(y_test, axis=1)
    for ensemble in save_ensembles :
        index_pred = ensemble.predict(x_test)  # 计算accuracy、PPV
        # Convert one-hot encoded test labels back to single class labels
        y_pred_labels = np.argmax(index_pred, axis=1)
        y_pred_labels_ensembles.append(y_pred_labels)
    # 按列投票，取每列中出现次数最多的类别作为最终分类结果
    final_result = mode(y_pred_labels_ensembles, axis=0, keepdims=False).mode.flatten()
    
    # 计算准确率
    accuracy = accuracy_score(y_test_labels, final_result)
    print(f'Accuracy: {accuracy:.2f}')
    
    # 打印分类报告
    print("Classification Report:")
    print(classification_report(y_test_labels, final_result))
    
    # 打印混淆矩阵
    print("Confusion Matrix:")
    print(confusion_matrix(y_test_labels, final_result))
if __name__ == "__main__":
    pop, stats,ensembles = main()
    
    print("##############################集成分类器的预测结果：################################")
    vote_ensembles(ensembles)

gen	evals	min            	max            
0  	40   	[0.8421 0.9719]	[0.8888 0.9826]
1  	40   	[0.8616 0.978 ]	[0.8888 0.9849]
2  	40   	[0.8644 0.9783]	[0.8906 0.9849]
3  	40   	[0.8736 0.9783]	[0.8906 0.985 ]
4  	40   	[0.876  0.9783]	[0.8929 0.9852]
##############################集成分类器的预测结果：################################
Accuracy: 0.86
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.97      0.74       437
           1       0.98      0.96      0.97       385
           2       0.92      0.82      0.87       284
           3       0.96      0.78      0.86       255
           4       0.89      0.82      0.86       259
           5       0.95      0.72      0.82       218
           6       0.99      0.91      0.95       260
           7       0.94      0.88      0.91       244
           8       0.95      0.69      0.80       211
           9       0.96      0.85      0.90       237

    accuracy                           0